# Initialization

In [1]:
import tqdm

In [2]:
import utixenon
import nton
import pickle

import glob

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)
import numpy as np
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mpldates
plt.style.use('/home/gvolta/.config/matplotlib/stylelib/giovanni.mplstyle')

import strax, straxen, cutax, wfsim, epix
straxen.print_versions(modules=('strax', 'straxen', 'cutax', 'wfsim', 'epix'))

*** Detector definition message ***
You are currently using the default XENON10 template detector.



Using nestpy version 1.5.0


,module,version,path,git
0,python,3.8.13,/opt/XENONnT/anaconda/envs/XENONnT_2022.06.3/bin/python,None
1,strax,1.2.3,/opt/XENONnT/anaconda/envs/XENONnT_2022.06.3/lib/python3.8/site-packages/strax,None
2,straxen,1.7.1,/opt/XENONnT/anaconda/envs/XENONnT_2022.06.3/lib/python3.8/site-packages/straxen,None
3,cutax,1.12.0,/dali/lgrandi/xenonnt/software/cutax/v1.12.0/cutax,None
4,wfsim,1.0.0,/opt/XENONnT/anaconda/envs/XENONnT_2022.06.3/lib/python3.8/site-packages/wfsim,None
5,epix,0.3.2,/home/gvolta/XENONnT/epix/epix,branch:update_giovanni | 3ccdd97


In [3]:
import uproot
from epix.simulator.fast_simulator import StraxSimulator

In [5]:
path = '/dali/lgrandi/giovo/XENONnT/epix/test/'
strax_data_dir = path
!ls {strax_data_dir}

fastsim_test.csv
fastsim_test-events_nveto-ybiznjvgl3
fastsim_test-events_tpc-ybiznjvgl3
nT_mc_run_ER_SS_InnerCryostat_U238_00000.root
nT_mc_WholeLXe_Xe127_04400.root
nT_mc_WholeLXe_Xe127_49999.root
nT_TPC_Pb212_100000_postpone0_0.root
nT_TPC_Pb212_2000_0.root
Xenon1T_ER_Copper_BottomPmtPlate_U235_00476_g4mc_G4p10.root
Xenon1T_ER_Copper_FieldGuard__Co60_00000_g4mc_G4p10.root
Xenon1T_ER_Copper_TopRing_Co60_00000_g4mc_G4p10.root
Xenon1T_ER_Copper_TopRing_Ra226_00000_g4mc_G4p10.root
Xenon1T_ER_Copper_TopRing_U238_00000_g4mc_G4p10.root


In [6]:
!rm -r /dali/lgrandi/giovo/XENONnT/epix/test/fastsim_test-*

In [7]:
!rm -r /dali/lgrandi/giovo/XENONnT/epix/test/wfsim_test*

rm: cannot remove ‘/dali/lgrandi/giovo/XENONnT/epix/test/wfsim_test*’: No such file or directory


In [8]:
!rm -r /dali/lgrandi/giovo/XENONnT/epix/test/*epix*

rm: cannot remove ‘/dali/lgrandi/giovo/XENONnT/epix/test/*epix*’: No such file or directory


In [9]:
raw_data_dir = path
raw_data_filename = 'nT_mc_WholeLXe_Xe127_04400.root'

In [10]:
g1_low_energy, g2_low_energy = utixenon.ces_helper.resolution_helper.ces_parameters(kind='xenonnt_v8_nominal_low_energy')

w = 13.7e-3

# Strax simulator

In [11]:
st = cutax.contexts.xenonnt_sim_chain_SR0v1_cmt_v8(output_folder=strax_data_dir) ##xenonnt_sim_SR0v3_cmt_v8 #xenonnt_sim_chain_SR0v1_cmt_v8

INFO:strax:Deregister records_mv
INFO:strax:Deregister detector_time_offsets
INFO:strax:Deregister hitlets_mv
INFO:strax:Deregister events_sync_mv
INFO:strax:Deregister events_sync_nv
INFO:strax:Deregister events_mv
INFO:strax:Deregister online_monitor_mv
INFO:strax:Replacing context._fixed_plugin_cache since plugins/versions changed


In [12]:
run_id = 'fastsim_test'

# epix config
epix_args={'path': raw_data_dir,
           'file_name': raw_data_filename, 
           'detector_config_override': '/home/gvolta/NucleonDisappearance/XENONnT/Simulation/EPIX/fastsim/sr0_epix_detectorconfig.ini',
           'debug': True,           
           'entry_start': 0,
           'entry_stop': None,
           'cut_by_eventid': False,
           'micro_separation_time': 10.0,
           'micro_separation': 0.05, # 0.005 mm
           'tag_cluster_by': 'energy',
           'max_delay': 1e7,
           'source_rate': -1,
           'save_epix': False}

In [13]:
configuration_files={'nv_pmt_qe':'nveto_pmt_qe.json',
                     'photon_area_distribution':'XENONnT_SR0_spe_distributions_20210713_no_noise_scaled.csv',
                     's1_pattern_map': 'XENONnT_s1_xyz_patterns_LCE_MCvf051911_wires.pkl',
                     's2_pattern_map': 'XENONnT_s2_xy_patterns_GXe_LCE_corrected_qes_MCv4.3.0_wires.pkl',
                    }

_gains = [1 for i in range(494)]       
# List of disabled PMTs
pmt_off = [28, 121, 156, 164, 177, 291, 313, 338, 350, 352, 354, 362, 386, 393, 405, 427, 461]
for _pmt in pmt_off:
    _gains[_pmt]=0
    
# Set up configuration
st.config.update(dict(fax_config = '/home/gvolta/XENONnT/private_nt_aux_files/sim_files/fax_config_nt_sr0_v3.json',  
                      fax_config_nveto = 'fax_config_nt_nveto.json',  
                      g4_file= path + raw_data_filename,
                      epix_config = epix_args,
                      # configuration_files = configuration_files,
                      fax_config_overrides = {# "detector": "XENONnT",
                                                # 'detector_config_override': '/dali/lgrandi/giovo/XENONnT/epix/sr0_epix_detectorconfig.ini',
                                              #  "gains": _gains,
                                                # "n_top_pmts": 253,
                                                # "n_tpc_pmts": 494,
                      }
                     ))

st.register(StraxSimulator)

epix.simulator.fast_simulator.StraxSimulator

# Run simualtion

In [14]:
import time
import os 
import shutil

start_time = time.time()

st.make(run_id,'events_tpc')
st.make(run_id, 'events_nveto')

t = time.time()
print(f'It took {(t - start_time):.1f} [s] to finish the simulation.\n\n')

INFO:strax:Replacing context._fixed_plugin_cache since plugins/versions changed
/home/gvolta/XENONnT/epix/epix/simulator/fast_simulator.py:203: UserWarning: Why we have to provide gains here? No gains are passed in fax_config_overrides, default equal to 1
  warnings.warn('Why we have to provide gains here? '
/home/gvolta/XENONnT/epix/epix/simulator/fast_simulator.py:207: UserWarning: This is a deault value, why we have to give it in fax_config_overrides? No n_top_pmts are passed in fax_config_overrides, default equal to 253
  warnings.warn('This is a deault value, why we have to give it in fax_config_overrides? '
/home/gvolta/XENONnT/epix/epix/simulator/fast_simulator.py:211: UserWarning: This is a deault value, why we have to give it in fax_config_overrides? No n_tpc_pmts are passed in fax_config_overrides, default equal to 494
  warnings.warn('This is a deault value, why we have to give it in fax_config_overrides? '
/opt/XENONnT/anaconda/envs/XENONnT_2022.06.3/lib/python3.8/site-pack

epix configuration:  {'path': '/dali/lgrandi/giovo/XENONnT/epix/test/', 'file_name': 'nT_mc_WholeLXe_Xe127_04400.root', 'detector_config_override': '/home/gvolta/NucleonDisappearance/XENONnT/Simulation/EPIX/fastsim/sr0_epix_detectorconfig.ini', 'debug': True, 'entry_start': 0, 'entry_stop': None, 'cut_by_eventid': False, 'micro_separation_time': 10.0, 'micro_separation': 0.05, 'tag_cluster_by': 'energy', 'max_delay': 10000000.0, 'source_rate': -1, 'save_epix': False, 'detector_config': [<epix.detector_volumes.SensitiveVolume object at 0x7fe4da47bd00>, <epix.detector_volumes.SensitiveVolume object at 0x7fe4d974b880>], 'outer_cylinder': {'max_z': 7.3936, 'min_z': -154.6555, 'max_r': 66.4}}
Total entries in input file = 2000
Starting to read from output file entry 0
It took 42.5430 sec to load data.
Finding clusters of interactions with a dr = 0.05 mm and dt = 10.0 ns
It took 11.1987 sec to find clusters.
It took 3.1705 sec to merge clusters.
Removing clusters not in volumes: TPC BelowCat

/home/gvolta/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:2387: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)


It took 1.3504 sec to get quanta.
Clean event separation


/opt/XENONnT/anaconda/envs/XENONnT_2022.06.3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Source finished!
It took 103.4 [s] to finish the simulation.




/home/gvolta/XENONnT/epix/epix/simulator/GenerateEvents.py:115: RuntimeWarning: invalid value encountered in true_divide
  instructions['cs1'] = instructions['s1_area'] / (resource.s1_map(event_positions)[:, 0]/0.1581797073725071)
/home/gvolta/XENONnT/epix/epix/simulator/GenerateEvents.py:116: RuntimeWarning: invalid value encountered in true_divide
  instructions['alt_cs1'] = instructions['alt_s1_area'] / (resource.s1_map(event_positions)[:, 0]/0.1581797073725071)


# Epix info

In [ ]:
!ls {raw_data_dir}

In [ ]:
epix_ins = np.load(raw_data_dir + 'nT_mc_WholeLXe_Xe127_49999_epix_1.npy')
epix_ins_2 = np.load(raw_data_dir + 'nT_mc_WholeLXe_Xe127_49999_epix_2.npy')

In [ ]:
epix_ins_tree = epix_ins.copy()
tree = pickle.load(open('/dali/lgrandi/jgrigat/s2_separation/s2_separation_decision_tree_fast_sim.p', 'rb+'))

In [ ]:
def _merge_these_clusters_nt_res_jaron(s2_area1, z1, s2_area2, z2, tree):
    return bool(tree.predict([[z1, z2-z1, s2_area1, s2_area2]]))

def macro_cluster_events(tree, instructions):
    """Loops over all instructions, checks if it's an s2 and if there is another s2 within the same event
        within the macro cluster distance, if it is they are merged."""
    for ix1, _ in enumerate(instructions):
        if instructions[ix1]['type'] != 2:
            continue
        for ix2 in range(1, len(instructions[ix1:])): # why was  + 1): ?
            if instructions[ix1 + ix2]['type'] != 2:
                continue
            if instructions[ix1]['event_number'] != instructions[ix1 + ix2]['event_number']:
                break
            # _nt_res
            if _merge_these_clusters_nt_res_jaron(instructions[ix1]['amp'], instructions[ix1]['z'],
                                                      instructions[ix1 + ix2]['amp'], instructions[ix1 + ix2]['z'], 
                                                      tree):
                #print(ix1)
                instructions[ix1 + ix2]['x'] = (instructions[ix1]['x'] + instructions[ix1 + ix2]['x']) * 0.5
                instructions[ix1 + ix2]['y'] = (instructions[ix1]['y'] + instructions[ix1 + ix2]['y']) * 0.5
                instructions[ix1 + ix2]['z'] = (instructions[ix1]['z'] + instructions[ix1 + ix2]['z']) * 0.5
                
                instructions[ix1 + ix2]['x_pri'] = (instructions[ix1]['x'] + instructions[ix1 + ix2]['x_pri']) * 0.5
                instructions[ix1 + ix2]['y_pri'] = (instructions[ix1]['y'] + instructions[ix1 + ix2]['y_pri']) * 0.5
                instructions[ix1 + ix2]['z_pri'] = (instructions[ix1]['z'] + instructions[ix1 + ix2]['z_pri']) * 0.5
                
                instructions[ix1 + ix2]['amp'] = int((instructions[ix1]['amp'] + instructions[ix1 + ix2]['amp']))
                instructions[ix1]['amp'] = -1  # flag to throw this instruction away later
                instructions[ix1 + ix2]['e_dep'] = (instructions[ix1]['e_dep'] + instructions[ix1 + ix2]['e_dep'])
                instructions[ix1]['e_dep'] = -1  # flag to throw this instruction away later
                break

In [ ]:
macro_cluster_events(tree, epix_ins_tree)

In [ ]:
epix_ins_df = pd.DataFrame(epix_ins) 
epix_ins_2_df = pd.DataFrame(epix_ins_2)
epix_ins_tree_df = pd.DataFrame(epix_ins_tree)

In [ ]:
epix_ins_tree_df = epix_ins_tree_df[epix_ins_tree_df.amp != -1]

In [ ]:
event_number_0 = []
event_number_1 = []
event_number_2 = []
for en in tqdm.tqdm(np.unique(epix_ins_tree_df.event_number)):
    
    temp = epix_ins_tree_df[epix_ins_tree_df.event_number == en] 
    
    if len(temp[temp.type == 2]) == 0:
        event_number_0.append(en)
    elif len(temp[temp.type == 2]) == 1:
        event_number_1.append(en)
    else:
        event_number_2.append(en)

In [ ]:
len(event_number_0), len(event_number_1), len(event_number_2)

In [ ]:
en = event_number_1[np.random.choice(len(event_number_1), 1)[0]]
display(epix_ins_df[(epix_ins_df.event_number == en) & (epix_ins_df.type == 2)]), display(epix_ins_tree_df[(epix_ins_tree_df.event_number == en) & (epix_ins_df.type == 2)])

In [ ]:
epix_ins_df

In [ ]:
epix_ins_2_df

In [ ]:
epix_ins_tree_df

# WFsim

In [ ]:
run_id_wfsim = 'wfsim_test'

st_sim = cutax.contexts.xenonnt_sim_SR0v2_cmt_v8()
#st_sim.set_config({'fax_config': '/home/gvolta/XENONnT/private_nt_aux_files/sim_files/fax_config_nt_sr0_v2.json'})
st_sim.set_config(dict(fax_file = raw_data_dir + 'nT_TPC_Pb212_2000_0_epix.csv'))

In [ ]:
st_sim.make(run_id_wfsim, 'raw_records')

In [ ]:
st_sim.make(run_id_wfsim, 'peaklets')
#st.make(run_id, 'event_pattern_fit')

In [ ]:
wfsim_event = st_sim.get_df(run_id_wfsim, 'event_info')

In [ ]:
wfsim_event['e_ces_low'] = w * (wfsim_event.cs1/g1_low_energy + wfsim_event.cs2/g2_low_energy)
wfsim_event['full_e_ces_low'] = w * (wfsim_event.cs1/g1_low_energy + (wfsim_event.cs2 + wfsim_event.alt_cs2)/g2_low_energy) 

In [ ]:
wfsim_event.head(5)

### Simulated events

In [ ]:
event_info = st.get_df(run_id, 'events_tpc')

event_info['r'] = np.sqrt(event_info['x']**2 + event_info['y']**2)
event_info['r_pri'] = np.sqrt(event_info['x_pri']**2 + event_info['y_pri']**2)

event_info['e_ces_low'] = w * (event_info.cs1/g1_low_energy + event_info.cs2/g2_low_energy)
event_info['full_e_ces_low'] = w * (event_info.cs1/g1_low_energy + (event_info.cs2 + event_info.alt_cs2)/g2_low_energy) 

event_info

In [ ]:
plt.figure(figsize=(8, 3), dpi=150)

plt.subplot(121)
utixenon.matplotlib_utils.plotXY(event_info.x, event_info.y, norm=mpl.colors.Normalize(), cmin=1, cmap='viridis')
plt.colorbar()

plt.subplot(122)
utixenon.matplotlib_utils.plotR2Z(event_info.r, event_info.z, norm=mpl.colors.Normalize(), cmin=1, cmap='viridis')
plt.colorbar()

plt.tight_layout(), plt.show()

In [ ]:
plt.figure(figsize=(8, 3), dpi=150)

plt.subplot(121)
utixenon.matplotlib_utils.plotXY(event_info.x_pri, event_info.y_pri, norm=mpl.colors.Normalize(), cmin=1, cmap='viridis')
plt.colorbar()

plt.subplot(122)
utixenon.matplotlib_utils.plotR2Z(event_info.r_pri, event_info.z_pri, norm=mpl.colors.Normalize(), ranges=((0, 7000), (-200, 120)), cmin=1, cmap='viridis')
plt.colorbar()

plt.tight_layout(), plt.show()

In [ ]:
def cut_s2_ss(s2):
    alt_s2 = (0.00153 * s2 + 150)   /(1 + np.exp( 5e-4 * (s2 - 30000))) + (0.00230 * s2 + 310)   /(1 + np.exp( 5e-4 * (30000 - s2)))
    return alt_s2

plt.figure(dpi=200)

plt.scatter(np.log10(event_info.s2_area), 
            np.log10(event_info.alt_s2_area), 
            s=1, label='Fast sim')
            #bins = (np.arange(2, 7, 0.01), 
            #        np.arange(1, 7, 0.01)), 
            #cmap='viridis', cmin=1, norm=mpl.colors.LogNorm())
#plt.colorbar()
plt.scatter(np.log10(wfsim_event.s2_area), 
            np.log10(wfsim_event.alt_s2_area), 
            s=1, label='WF sim')

s2 = np.arange(1, 10000000, 500)
plt.plot(np.log10(s2), np.log10(cut_s2_ss(s2)), color='red', ls='--', label='Current S2 SS cut')

plt.xlabel('log$_{10}$(S2) [PE]'), plt.ylabel('log$_{10}$(S2$_{alt}$) [PE]')
plt.legend(loc='upper left')
plt.show()

In [ ]:
def get_cS2(cS1, ces, g1, g2, W = 13.7e-3):
    return g2 * (ces/W - cS1/g1)

plt.figure(figsize=(15, 3), dpi=150)

plt.subplot(131)
utixenon.matplotlib_utils.plotXY(event_info.x, event_info.y, norm=mpl.colors.Normalize(), cmin=1, cmap='viridis')
plt.colorbar()

plt.subplot(132)
utixenon.matplotlib_utils.plotR2Z(event_info.r, event_info.z, norm=mpl.colors.Normalize(), cmin=1, cmap='viridis')
plt.colorbar()

plt.subplot(133)
plt.hist2d(event_info.cs1, np.log10(event_info.cs2), 
           bins=(np.arange(0, 60000, 10), np.arange(2.9, 6.0, 0.02)), norm=mpl.colors.Normalize(), cmin=1)
#plt.hist2d(event_info.s1_area, np.log10(event_info.s2_area), 
#           bins=(np.arange(0, 6000, 10), np.arange(2.9, 6.0, 0.02)), norm=mpl.colors.Normalize(), cmin=1, cmap='inferno')
#plt.colorbar()

#cS1 = np.arange(0, 1000000, 1)
#for ces in [41.5, 163.9, 236.2, 609.3, 1173.2, ]:
## plt.plot(np.log10(cS1), np.log10(get_cS2(cS1, ces, g1_low_energy, g2_low_energy)), color='black', ls='--')
#    plt.plot(cS1, get_cS2(cS1, ces, g1_high_energy, g2_high_energy)/100, color='black', ls='--')

plt.xlabel('cS1 [PE]'), plt.ylabel('log10(cS2) [PE]')


plt.tight_layout(), plt.show()

In [ ]:
# epix_e_dep, bins = np.histogram(epix_ins[epix_ins.e_dep > 5].e_dep, bins=np.arange(0, 600, 4))

plt.figure(figsize=(8, 3), dpi=150)

plt.subplot(121)
plt.hist(event_info.e_ces_low, bins=np.arange(0, 600, 4))
plt.hist(wfsim_event.e_ces_low, bins=np.arange(0, 600, 4))
# plt.plot(0.5*(bins[1:] + bins[:-1]), epix_e_dep/2, ds='steps')
plt.xlabel('CES [keV]')
plt.grid()
#plt.yscale('log')

plt.subplot(122)
plt.hist(event_info.full_e_ces_low, bins=np.arange(0, 600, 4))
plt.hist(wfsim_event.full_e_ces_low, bins=np.arange(0, 600, 4))
#plt.yscale('log')
plt.xlabel('CES [keV]')
plt.grid()


plt.tight_layout(), plt.show()

In [ ]:
path = '/home/gvolta/'
!ls {path}

In [ ]:
spe_distribution_temp = np.load(path + 'spe_distribution_temp.npy')
scaled_bins_temp = np.load(path + 'scaled_bins_temp.npy')

In [ ]:
spe_shapes = pd.read_csv('/home/gvolta/XENONnT/private_nt_aux_files/sim_files/XENONnT_SR0_spe_distributions_20210713_no_noise_scaled.csv')

In [ ]:
uniform_to_pe_arr = []
for ch in spe_shapes.columns[1:]:  # skip the first element which is the 'charge' header
    if spe_shapes[ch].sum() > 0:
        # mean_spe = (spe_shapes['charge'].values * spe_shapes[ch]).sum() / spe_shapes[ch].sum()
        scaled_bins = spe_shapes['charge'].values  # / mean_spe
        cdf = np.cumsum(spe_shapes[f'{ch}']) / np.sum(spe_shapes[f'{ch}'])
    else:
        # if sum is 0, just make some dummy axes to pass to interpolator
        print(ch)
        cdf = np.linspace(0, 1, 10)
        scaled_bins = np.zeros_like(cdf)

    grid_cdf = np.linspace(0, 1, 2001)
    grid_scale = sp.interpolate.interp1d(cdf, scaled_bins,
                                         bounds_error=False,
                                         fill_value=(scaled_bins[0], scaled_bins[-1]))(grid_cdf)

    uniform_to_pe_arr.append(grid_scale)

In [ ]:
plt.figure(dpi=150)

plt.plot(np.linspace(0, 1, 2001), spe_distribution_temp)
plt.plot(np.linspace(0, 1, 2001), np.mean(uniform_to_pe_arr, axis=0))

plt.grid()
plt.show()

In [ ]:
s1_area_spe = []
for n_ph in :
    s1_area_spe.append(np.sum(spe_distribution[
                                  (np.random.random(n_ph) * len(spe_distribution)).astype(np.int64)]))
        return np.array(s1_area_spe)np.sum(spe_distribution_temp[(np.random.random(n_ph) * len(spe_distribution_temp)).astype(np.int64)])

In [ ]:
ch = 0
plt.figure(dpi=150)

plt.plot(spe_shapes['charge'].values, np.cumsum(spe_shapes[f'{ch}']) / np.sum(spe_shapes[f'{ch}']))

plt.grid()
plt.show()